This tutorial aims to run IGM providing an RGI ID

In [2]:
#!/usr/bin/env python3
 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math
import igm

 ## add custumized smb function
def params_mysmb(parser):  
    parser.add_argument("--meanela", type=float, default=3000 )

def init_mysmb(params,state):
    params.meanela = np.quantile(state.usurf[state.thk>10],0.2)

def update_mysmb(params,state):
    # perturabe the ELA with sinusional signal 
    ELA = ( params.meanela - 750*math.sin((state.t/100)*math.pi) )
    # compute smb linear with elevation with 2 acc & abl gradients
    state.smb  = state.usurf - ELA
    state.smb *= tf.where(state.smb<0, 0.005, 0.009)
    # cap smb by 2 m/y 
    state.smb  = tf.clip_by_value(state.smb, -100, 2)
    # make sure the smb is not positive outside of the mask to prevent overflow
    if hasattr(state, "icemask"):
        state.smb  = tf.where((state.smb<0)|(state.icemask>0.5),state.smb,-10)
    
def final_mysmb(params,state):
    pass

# make sure to make these function new attributes of the igm module
igm.params_mysmb  = params_mysmb  
igm.init_mysmb    = init_mysmb  
igm.update_mysmb  = update_mysmb
igm.final_mysmb   = final_mysmb
 
# Select one OPTION btw the first, keep the MANDATORY ones, un/comment OPTIONAL modules
modules = [
           "prepare_data",          # OPTION 1  : download and prepare the data with OGGM
#            "optimize",             # OPTIONAL  : optimize unobservable variables from obs.
            "mysmb",                # OPTIONAL  : custom surface mass balance model
            "flow_dt_thk",          # MANDATORY : update the ice thickness solving mass cons.
            "vertical_iceflow",     # OPTIONAL  : retrieve vertical ice flow from horiz.
#           "particles",            # OPTIONAL  : seed and update particle trajectories
#           "write_particles",      # OPTIONAL  : write particle trajectories to a csv file
            "write_ncdf_ex",        # OPTIONAL  : write 2d state data to netcdf files
            "write_tif_ex",        # OPTIONAL  : write the result in tif files
#           "write_ncdf_ts",        # OPTIONAL  : write time serie data to netcdf files
#           "write_plot2d",         # OPTIONAL  : write 2d state plots to png files
            "print_info",           # OPTIONAL  : print basic live-info about the model state
            "print_all_comp_info",  # OPTIONAL  : report information about computation time
            "anim3d_from_ncdf_ex"   # OPTIONAL  : make a nice 3D animation of glacier evolution
          ]

# Collect and parse all the parameters of all model components
parser = igm.params_core()
for module in modules:
    getattr(igm, "params_" + module)(parser)
params = parser.parse_args(args=[])

# Override parameters
params.RGI         = 'RGI60-11.01238'
params.tstart      = 2000.0
params.tend        = 2100.0
params.tsave       = 5
params.plot_live   = True
params.observation = ("optimize" in modules)

# Define a state class/dictionnary that contains all the data
state = igm.State()
igm.init_state(params, state)

# Place the computation on your device GPU ('/GPU:0') or CPU ('/CPU:0')
with tf.device("/GPU:0"):

    # Initialize all the model components in turn
    for module in modules:
        getattr(igm, "init_" + module)(params, state)

    # Time loop, perform the simulation until reaching the defined end time
    while state.t < params.tend:
        
        # Update each model components in turn
        for module in modules:
            getattr(igm, "update_" + module)(params, state)
            
    # Finalize each module in turn
    for module in modules:
        getattr(igm, "final_" + module)(params, state)


2023-07-26 08:53:51: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2023-07-26 08:53:51: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2023-07-26 08:53:51: oggm.cfg: Multiprocessing: using all available processors (N=16)


PARAMETERS ARE ...... 
                   working_dir : 
                 logging_level : CRITICAL
                  logging_file : igm.log
                           RGI : RGI60-11.01238
                    preprocess : True
                            dx : 100
                        border : 30
                    thk_source : consensus_ice_thickness
                   observation : False
                 path_glathida : /home/gjouvet/
                output_geology : True
                       meanela : 3000
                  type_iceflow : emulated
                      emulator : f21_pinnbp_GJ_23_a
               iceflow_physics : 2
                init_slidingco : 10000
                init_arrhenius : 78
                     regu_glen : 1e-05
                 regu_weertman : 1e-10
                      exp_glen : 3
                  exp_weertman : 3
                            Nz : 10
                  vert_spacing : 4.0
                   thr_ice_thk : 0.1
       solve_iceflo

2023-07-26 08:53:55: oggm.workflow: init_glacier_directories from prepro level 2 on 1 glaciers.
2023-07-26 08:53:55: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 1 glaciers


2023-07-26 08:53:55: tensorflow: No training configuration found in the save file, so the model was *not* compiled. Compile it manually.


IGM %s :         Iterations   |         Time (y)     |     Time Step (y)   |   Ice Volume (km^3) 


2023-07-26 08:53:56.991684: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8800
2023-07-26 08:53:58.370843: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-07-26 08:53:58.373550: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-07-26 08:53:58.373611: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-07-26 08:53:58.375733: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-07-26 08:53:58.375955: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-07-26 08:53:58.510281: I tensorflow/stream_executor/cuda/c

IGM 08:54:07 :          11    |          2005        |        0.63        |           1.61 
IGM 08:54:07 :          19    |          2010        |        0.62        |           1.72 
IGM 08:54:08 :          30    |          2015        |        0.43        |           1.84 
IGM 08:54:08 :          46    |          2020        |        0.30        |           1.95 
IGM 08:54:09 :          62    |          2025        |        0.39        |           2.04 
IGM 08:54:09 :          75    |          2030        |        0.43        |           2.13 
IGM 08:54:09 :          87    |          2035        |        0.43        |           2.21 
IGM 08:54:10 :         100    |          2040        |        0.39        |           2.28 
IGM 08:54:10 :         116    |          2045        |        0.26        |           2.33 
IGM 08:54:11 :         141    |          2050        |        0.19        |           2.37 
IGM 08:54:11 :         168    |          2055        |        0.19        |     